In [ ]:
import corus
from corus import load_mokoron

## 1. Я выбрала следующие пять неоднозначнх лексем: лук, таз, такт, тон, торчать. Значения брала из большого толкового словаря Кузнецова: https://gufo.me/dict/kuznetsov

In [2]:
words_dictionary = {'лук':[
                        'Огородное растение семейства лилейных.',
                        'Съедобные трубчатые листья или луковицы этого растения.',
                        'Ручное оружие для метания стрел, изготовленное из гибкого, упругого стержня (обычно деревянного), стянутого в дугу тетивой.'],
                    'таз':[
                        'Широкий и неглубокий округлый металлический сосуд.',
                        'Часть скелета человека и животных - костный пояс, опирающийся на нижние (у животных - на задние) конечности и служащий опорой для позвоночника.'],
                    'такт':[
                        'Метрическая музыкальная единица - музыкальное движение между двумя соседними ударными моментами (долями).',
                        'Ритм какого-л. движения, действия; каждое из равномерно следующих одно за другим движений или ударов.',
                        'Часть рабочего цикла какого-л. механизма.',
                        'Чувство меры в поведении, в поступках; умение вести себя пристойно.'],
                    'тон':[
                        'Музыкальный звук определённой высоты в отличие от шума.',
                        'Соотношение двух звуков темперированного звукоряда, принимаемое за единицу при определении разности звуков по высоте и равное двум полутонам.',
                        'Звук работающего сердца, его клапанов; звук, получаемый при выстукивании полых органов человеческого тела.',
                        'Характер, оттенок звучания инструмента или голоса.',
                        'Манера, стиль общения, письма, повествования.',
                        'Цвет, окраска, а также оттенок какого-л. цвета, отличающийся той или иной степенью яркости, насыщенности.',
                        'Косметическое средство, накладываемое на лицо для усиления выразительности лица, скрадывания изъянов и т.п.',
                        'Звук аккорда, совпадающий с первой ступенью гаммы и слышимый отчётливее, чем другие.',
                        'Самый низкий звук, получаемый от колебания звучащего тела, слышимый лучше других и обозначаемый обычно как единственно звучащий.'],
                    'торчать':[
                        'Возвышаться, выступать над поверхностью чего-л., выдаваться вперёд, вверх.',
                        'Стоять, размещаться где-л.',
                        'Присутствовать, находиться где-л.',
                        'Испытывать удовольствие от чего-л.']}

# 2. Скачиваю датасет постов из твиттера

In [ ]:
records = load_mokoron('nlp/twitter_db.sql')
next(records)

MokoronRecord(
    id=408906695721877504,
    timestamp=datetime.datetime(2013, 12, 6, 10, 32, 8),
    user='Va5ilina',
    text='Пропавшая в Хабаровске школьница почти сутки провела в яме у коллектор',
    sentiment=2,
    replies=0,
    retweets=0,
    favourites=0,
    posts=183,
    followers=95,
    friends=158,
    lists=0
)

In [ ]:
original_texts = []
for i, record in enumerate(records):
    original_texts.append(record.text)
    if i == 1999999:
        break
print(len(original_texts), original_texts[0])

2000000 ЛЕНТА, Я СЕГОДНЯ ПОЛГОДА ДИРЕКШИОНЕЕЕЕР! С:\nХОТЯ ВСЕ РАВНО НИКТО НЕ ПОЗДРАВИТ ЛОЛ


### Чтобы не уснуть, пока всё это будет лемматизироваться, беру только часть датасета (4 млн твитов). Затем склеиваю по 10 000 текстов в одну строку (вставляя между ними разделитель 'ddeelliimmiitteerr'). Это ускорит работу лемматизатора (идею взяла вот тут: https://habr.com/ru/articles/503420/). Потом списки токенов я обратно порежу на исходные тексты с помощью всё того же разделителя. У меня получается 200 таких тексто-колбасок, они сложены в список long_texts

In [ ]:
long_text_size = 10000
chunks = [original_texts[i:i + long_text_size] for i in range(0, len(original_texts), long_text_size)]
long_texts = []
for chunk in chunks:
    long_texts.append(' ddeelliimmiitteerr '.join(chunk))

print(long_texts[0][0:200]) #вот так выглядит кусочек строки из склеенных через разделитель текстов

ЛЕНТА, Я СЕГОДНЯ ПОЛГОДА ДИРЕКШИОНЕЕЕЕР! С:\nХОТЯ ВСЕ РАВНО НИКТО НЕ ПОЗДРАВИТ ЛОЛ ddeelliimmiitteerr Царствие Божие внутрь вас есть. ddeelliimmiitteerr RT @TwitRegion: Ученые: кофе приносит намного б


### Для лемматизации я буду использовать mystem

In [4]:
import nltk
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import time
from tqdm import tqdm

In [5]:
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [ ]:
stopwords = list(stopwords.words('russian'))
stopwords.extend(['\n', ' ', '  ', '', 'http', '://', 't', 'co'])
stopwords.extend(punctuation)
print(stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [ ]:
def preprocessing(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token.strip() not in stopwords]
    return tokens

In [ ]:
tokenized_texts = []
for text in tqdm(long_texts):
    tokenized_texts.append(preprocessing(text))
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [28:43<00:00,  8.62s/it]


In [ ]:
len(tokenized_texts)

200

In [ ]:
print(tokenized_texts[0][0:100]) #вот так выглядит кусочек списка токенов

['лента', 'сегодня', 'полгода', 'дирекшионеееер', ':\\', 'пхотить', 'равный', 'никто', 'поздравлять', 'лола', 'ddeelliimmiitteerr', 'царствие', 'божий', 'внутрь', 'ddeelliimmiitteerr', 'rt', 'twitregion', 'ученый', 'кофе', 'приносить', 'намного', 'вред', 'пиво', 'ddeelliimmiitteerr', 'zzzwylsmon', 'справка', 'адаптация', '5', 'классник', 'ddeelliimmiitteerr', 'взаимныйфоловинг', 'ddeelliimmiitteerr', 'rt', 'v1ka', 'si', 'удивлять', 'маленький', 'девочка', 'который', 'нагло', 'писать', 'знакомиться', 'говорить', 'поздороваться', 'завтра', 'френд', '…', 'ddeelliimmiitteerr', 'rt', 'digger2912', ': \\"', 'угол', 'сидеть', 'погибать', 'голод', '2', 'порция', 'взять', 'хотя', 'жрать', 'хотеть', '\\" :', 'dd', 'gqg6iue2', '…', 'ddeelliimmiitteerr', '7ed9e1k62c', 'виленкин', '6', 'класс', '2006', 'год', 'ответ', 'n', '7ed9e1k62c', 'ddeelliimmiitteerr', 'england', 'artie', 'все-таки', 'сделать', 'это', 'ddeelliimmiitteerr', 'rt', 'yowzah', '_: ', 'ммм', 'ппирожок', 'ловетта', 'пиха', 'обожать'

In [ ]:
def splitter(text, delimiter = 'ddeelliimmiitteerr'):
    splitted = [[]]
    for item in text:
        if item == delimiter:
            splitted.append([])
        else:
            splitted[-1].append(item)
    return splitted

In [ ]:
print(splitter(tokenized_texts[0])[0])

['лента', 'сегодня', 'полгода', 'дирекшионеееер', ':\\', 'пхотить', 'равный', 'никто', 'поздравлять', 'лола']


In [ ]:
short_tokenized_texts = []
for long_text in tqdm(tokenized_texts):
    short_tokenized_texts.extend(splitter(long_text))
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [03:35<00:00,  1.08s/it]


In [ ]:
short_tokenized_texts[0]

['лента',
 'сегодня',
 'полгода',
 'дирекшионеееер',
 ':\\',
 'пхотить',
 'равный',
 'никто',
 'поздравлять',
 'лола']

Оно очень долго лемматизировалось, поэтому сохраняю на всякий случай в файлик

In [3]:
import csv

In [ ]:
with open('short_tokenized_texts', 'w', encoding='utf-8', newline='') as file:
    writer = csv.writer(file, delimiter = ',')
    for text in short_tokenized_texts:
        writer.writerow(text)

In [ ]:
with open('original_texts.txt', 'w', encoding='utf-8') as file:
    for text in original_texts:
        file.write(text)
        file.write('textdelimitertext')

In [25]:
short_tokenized_texts = []
with open('short_tokenized_texts', 'r', encoding='utf-8', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    for line in reader:
        short_tokenized_texts.append(list(line))

In [27]:
original_texts = []
with open('original_texts.txt', 'r', encoding='utf-8', newline='') as file:
    full = file.read()
    original_texts = full.split('textdelimitertext')

In [28]:
print(original_texts[48394], len(original_texts))
print(short_tokenized_texts[48394], len(short_tokenized_texts))

лишь перенесение этих чувств на наши чувства к отдельным людям. http://t.co/1V3EGqhrDr 2000001
['лишь', 'перенесение', 'чувство', 'наш', 'чувство', 'отдельный', 'человек', '1v3egqhrdr'] 2000000


In [30]:
original_texts = original_texts[0:2000000] #где-то лажанула с циферками, проще всего исправить вот так :)
print(len(original_texts))

2000000


In [31]:
words = words_dictionary.keys()

In [54]:
def word_finder(word):
    texts_w_word = []
    for idx, text in enumerate(short_tokenized_texts):
        if word in text:
            texts_w_word.append([original_texts[idx], short_tokenized_texts[idx]])
    return texts_w_word

In [55]:
word_entries = {}
for word in words:
    word_entries[word] = word_finder(word)

In [56]:
for key, value in word_entries.items():
    print(key, value[:5], '\n')

лук [['RT @naminyvapaq: Без лука ты никто. Обзор 5-й недели в НБА', ['rt', 'naminyvapaq', 'лук', 'никто', 'обзор', '5', 'й', 'неделя', 'нба']], ['Без лука ты никто. Обзор 5-й недели в НБА', ['лук', 'никто', 'обзор', '5', 'й', 'неделя', 'нба']], ['@ausyashnik_eeeh обычный подъезд, у нас соседи-выблядки за стеной лук всегда жарили, так через розетки все к нам шло, розетку залепили в пзд', ['ausyashnik', 'eeeh', 'обычный', 'подъезд', 'сосед', 'выблядок', 'стена', 'лук', 'жарить', 'розетка', 'идти', 'розетка', 'залеплять', 'пзд']], ['а если макарошки отварить и запечь с луком,морковкой,майонезом,чесноком и всякими приправками,будет ОМНОМНОМ! *О* сыра нет,жаль(', ['макарошка', 'отваривать', 'запекать', 'лук', 'морковка', 'майонез', 'чеснок', 'всякий', 'приправка', 'омномный', 'сыр', 'жаль']], ['RT @wunopanivos: Без лука ты никто. Обзор 5-й недели в НБА', ['rt', 'wunopanivos', 'лук', 'никто', 'обзор', '5', 'й', 'неделя', 'нба']]] 

таз [['знакомая сфоткалась в тазу, в тряпке, сделала руку ка

# 3. Adagram

In [42]:
pip install Cython numpy

In [35]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-ldvfdq0i
  Running command git clone --filter=blob:none --quiet https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-ldvfdq0i
  Resolved https://github.com/lopuhin/python-adagram.git to commit cf3639f10d6a1efbcb602f45a1da89ef55ce5794
  Preparing metadata (setup.py) ... done
  Created wheel for adagram: filename=adagram-0.0.1-cp310-cp310-linux_x86_64.whl size=686336 sha256=90baff7369ce459256151ee89f9817b6f98ea8f54c5d1b6477919049bf7db9d4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ylre99cg/wheels/88/8b/fc/291709600177e98c9bba397433476b87d894ca506e78302432
Successfully built adagram


In [37]:
!curl "https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib" > all.a010.p10.d300.w5.m100.nonorm.slim.joblib

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1394M  100 1394M    0     0  44.2M      0  0:00:31  0:00:31 --:--:-- 47.1M


In [169]:
import adagram
import pandas as pd
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [171]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [172]:
for word in words:
    w_sense_p = vm.word_sense_probs(word)
    print(word, w_sense_p)

лук [(0, 0.27467387333364635), (1, 0.17510064513645113), (2, 0.25182447428150506), (3, 0.0852764349918363), (4, 0.2131232975158767)]
таз [(0, 0.3090465121813698), (1, 0.5411736692478084), (2, 0.010935272052302873), (3, 0.13883941377195283)]
такт [(0, 0.5839807974058925), (1, 0.22251124185169976), (2, 0.19350381193147648)]
тон [(0, 0.5003589738557619), (1, 0.19110443705896477), (2, 0.07183727112253462), (3, 0.2366986071800232)]
торчать [(0, 0.2462484294440683), (1, 0.18975992258666605), (2, 0.2858422026348257), (3, 0.2383622314186124), (4, 0.0397857781051299)]


In [173]:
def context_probability(word):
    '''
    предсказывает значение слова в определенном контексте
    применяется в цикле по всем контекстам для каждого слова (ниже)
    '''
    word_contexts = {}
    contexts = word_entries[word]
    for orig, lemmas in contexts:
        sent_list = [lemma for lemma in lemmas if lemma != word]
        means = vm.disambiguate(word, sent_list)
        word_contexts[orig] = np.argmax(means)
    return word_contexts

In [174]:
'''
adagram_predictions - словарь, в котором каждому слову соответсвует большой список.
каждый элемент этого большого списка - списочек из оригинального и лемматизированного контекстов
'''
adagram_predictions = {}
for word in words:
    adagram_predictions[word] = context_probability(word)

/usr/local/lib/python3.10/dist-packages/adagram/model.py:171: RuntimeWarning: divide by zero encountered in log
  z = np.log(z)


In [ ]:
'''
этот цикл проходится по словарным определениям значений и присваивает им нпредсказания модели
'''
dictionary_predictions = {}
for article in words_dictionary.items():
    meaning_predictions = {}
    word, meanings = article
    for meaning in meanings:
        meaning_tokens = mystem.lemmatize(meaning.lower())
        means = vm.disambiguate(word, meaning_tokens)
        meaning_predictions[meaning] = np.argmax(means)
    dictionary_predictions[word] = meaning_predictions
dictionary_predictions

#### Предположим, я выбрала следующие значения:
    лук:
    1) огородное растение - 0
    2) оружие - 4

    таз:
    1) медицинсоке понятие - 0

    такт:
    1) метрическая муз. единица - 2
    2) чувство меры в поведении - 1

    тон:
    1) муз. звук определённой высоты - 2
    2) манера, стиль общения, письма, повествования - 3
    3) оттенок - 1

    торчать:
    1) возвышаться, выступать над поверхностью - 0
    2) находиться где-либо - 3

In [176]:
new_words_dictionary = {'лук': [0, 4], 'таз': [0], 'такт': [1, 2], 'тон': [1, 2, 3], 'торчать': [0, 3]}

In [177]:
'''
две функции ниже позволя.т выбрать пять контекстов, которым модель сделала те же предсказания,
что и выбранным мной толкованиям из словаря
'''
def contexts_finder(word):
    contexts_for_evaluation = []
    for meaning_id in new_words_dictionary[word]:
        five_contexts = sub_contexts_finder(word, meaning_id)
        contexts_for_evaluation.append(five_contexts)
    result = {}
    result[word] = contexts_for_evaluation
    return result

def sub_contexts_finder(word, mean_id):
    five_contexts = []
    for context, pred_id in adagram_predictions[word].items():
        if len(five_contexts) < 5:
            if pred_id == mean_id:
                five_contexts.append(context)
    result = {}
    result[mean_id] = five_contexts
    return result

In [180]:
for word in new_words_dictionary.keys():
    print(contexts_finder(word))

{'лук': [{0: ['http://t.co/Q7VohWcXKM внешки для лука в pw', 'Салат из редиски с сыром, яблоком и зеленым луком. | Рецепты салатов и закусок http://t.co/cuZWtKxSOX', '#кулинарное Овощной плов с перцем: Ингредиенты: 2 стакана риса, 3 сладких перца, 4 шт. репчатых лука, 1 ст.... http://t.co/KvLf5SAslB #едим', 'http://t.co/00U1aNO4ZY внешки для лука в pw\\n http://t.co/00U1aNO4ZY', '@PRroom курица лук вода!:)) http://t.co/tpDOSxdO9s']}, {4: ['нашла крутого блоггера. в каждом луке присутствует черное, никогда не видела такого кол-ва черной одежды', 'RT @bebimynikage: Найду себе лука, стану Купидоном... Ну тогда держитесь...', '#лук Красивые девушки на каблуках', '@faggot_sykes если ты мне сейчас не ответишь, я выстрелю в тебя из лука аки огненная Китнисс :-)', 'какой уровень нужен чтобы зачаровать лук на бесконечность http://t.co/Nlj6nl6lCT']}]}
{'таз': [{0: ['узкий таз.акушерство.национальное руководство.аламазян http://t.co/iNiN9A1RXR', 'Бейбiт Ыстыбаев таза женiс! Kaz 4 - Mgl 1.  Жеңісі

In [181]:
'''
ручками пишу новый словарь, в котором будет такая структура:

{
    word:[0, 1, 1, 0, 0],
    word:[0, 1, 0, 1, 1, 1, 0, 0, 1, 1], ...
}
'''

'\nручками пишу новые похожие на сверху словари, в которых будет такая структура:\n{\n    word: (предсказание модели, мое предсказание)\n}\n'

In [182]:
to_evaluate = {
    'лук': [0, 1, 1, 0, 1, 0, 1, 0, 1, 1],
    'таз': [1, 1, 1, 1, 1],
    'такт': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'тон': [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0],
    'торчать': [1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
}

In [185]:
def accuracy(eval_dict):
    words_accuracies = []
    for word, evals in eval_dict.items():
        word_acc = {}
        word_acc[word] = (sum(evals)/len(evals))
        words_accuracies.append(word_acc)
    return words_accuracies

In [186]:
accuracy(to_evaluate)

[{'лук': 0.6},
 {'таз': 1.0},
 {'такт': 1.0},
 {'тон': 0.4666666666666667},
 {'торчать': 0.9}]